In [1]:
import pathlib
import pickle
import zipfile

import numpy as np
import pandas as pd

# Parse SBAC results

In [2]:
cwd = pathlib.Path.cwd()
data_path = cwd / 'data'
csv_path = data_path / 'csv'
zip_path = data_path / 'zip'
pkl_path = data_path / 'pkl'

if not zip_path.exists():
    raise FileNotFoundError(f"'{zip_path}' does not exist")
    
if not pkl_path.exists():
    raise FileNotFoundError(f"'{pkl_path}' does not exist")

pathlib.Path.mkdir(csv_path, exist_ok=True)

In [10]:
CDS_CODE = ['County Code', 'District Code', 'School Code']

def read_zip(zip_file):
    with zipfile.ZipFile(zip_file) as zf:
        results_file, entities_file = zf.filelist
        results_csv = zf.extract(results_file, csv_path)
        entities_csv = zf.extract(entities_file, csv_path)
    
    #: Results Cleanse
    results_df = (pd.read_csv(results_csv)
                    .set_index(CDS_CODE)
                    # all `Test Types` = B; `Subgroup IDs` = 1; `Filler` is blank
                    .drop(columns=['Filler', 'Test Type', 'Subgroup ID'])  
                    .replace('*', np.nan)
                    .astype(float))

    pcts = [col for col in results_df.columns if 'Percent' in col]
    results_df.loc[:,pcts] = results_df.loc[:,pcts].applymap(lambda x: float(x) / 100)
    
    results_df.index.names = list('CDS')
    results_df.columns = results_df.columns.str.replace(' ', '_')

    #: Entities Cleanse
    entities_df = (pd.read_csv(entities_csv, encoding='latin-1')
                     .set_index(CDS_CODE)
                     .drop(columns=['Filler'])
                     .replace(' ' * 5, np.nan))
    
    entities_df.index.names = list('CDS')
    entities_df.columns = 'Year Type_Id County District School Zip'.split()
    entities_df = entities_df.drop(columns=['Year'])
                   
    return results_df, entities_df

In [11]:
file_2018 = zip_path / 'sb_ca2018_1_csv_v3.zip'
res, ent = results_2018, entities_2018 = read_zip(file_2018)

In [12]:
res.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 101725 entries, (0, 0, 0) to (58, 72769, 5838305)
Data columns (total 26 columns):
Test_Year                            101725 non-null float64
Total_Tested_At_Entity_Level         95556 non-null float64
Total_Tested_with_Scores             95526 non-null float64
Grade                                101725 non-null float64
Test_Id                              101725 non-null float64
CAASPP_Reported_Enrollment           89436 non-null float64
Students_Tested                      89436 non-null float64
Mean_Scale_Score                     68472 non-null float64
Percentage_Standard_Exceeded         89436 non-null float64
Percentage_Standard_Met              89436 non-null float64
Percentage_Standard_Met_and_Above    89436 non-null float64
Percentage_Standard_Nearly_Met       89436 non-null float64
Percentage_Standard_Not_Met          89436 non-null float64
Students_with_Scores                 89436 non-null float64
Area_1_Percentage_Above_

In [13]:
res.head()
res.dropna(thresh=6).shape

(89436, 26)

# To get Zip Codes to scrape

In [14]:
zip_code_arr = ent['Zip'].dropna().map(int).unique()
zip_code_path = csv_path / 'zip_codes.csv'
np.savetxt(zip_code_path, zip_code_arr, delimiter=",")

# Get results of scraped Zip Codes into DataFrame

In [15]:
zip_code_pkl = pkl_path / 'zip_codes.pkl'
with open(zip_code_pkl, 'rb') as fp:
    zip_code_dct = pickle.load(fp)
    
zip_code_dct = {str(k): v.get('Median Household Income', np.nan) for k, v in zip_code_dct.items()}
ent['Median_Income'] = ent['Zip'].map(zip_code_dct)
df = res.join(ent, how='inner').drop(columns=['County', 'District', 'School'])
df = df[df['Grade'] != 13]
df = df[df['CAASPP_Reported_Enrollment'].notnull()]
df = df[df['Median_Income'].notnull()]

# Store cleansed data with target and features

In [16]:
df_pkl = pkl_path / 'observations.pkl'
df.to_pickle(df_pkl)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 56928 entries, (1, 10017, 112607) to (58, 72769, 5838305)
Data columns (total 29 columns):
Test_Year                            56928 non-null float64
Total_Tested_At_Entity_Level         56928 non-null float64
Total_Tested_with_Scores             56928 non-null float64
Grade                                56928 non-null float64
Test_Id                              56928 non-null float64
CAASPP_Reported_Enrollment           56928 non-null float64
Students_Tested                      56928 non-null float64
Mean_Scale_Score                     56928 non-null float64
Percentage_Standard_Exceeded         56928 non-null float64
Percentage_Standard_Met              56928 non-null float64
Percentage_Standard_Met_and_Above    56928 non-null float64
Percentage_Standard_Nearly_Met       56928 non-null float64
Percentage_Standard_Not_Met          56928 non-null float64
Students_with_Scores                 56928 non-null float64
Area_1_Percentage_A